# Detection with SSD

In this example, we will load a SSD model and use it to detect objects.

### 1. Setup

* First, Load necessary libs and set up caffe and caffe_root

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

plt.rcParams['figure.figsize'] = (10, 10)
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# Make sure that caffe is on the python path:
caffe_root = '/home/user/caffe-ssd-nexar'
import os
import sys
sys.path.insert(0, os.path.join(caffe_root, 'python'))

In [ ]:
!nvidia-smi

In [ ]:
import caffe
caffe.set_device(1)
caffe.set_mode_gpu()

* Disable scrolling output

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

* Load LabelMap.

In [ ]:
from google.protobuf import text_format
from caffe.proto import caffe_pb2

# load Nexar labels
labelmap_file = os.path.join(caffe_root, 'data/nexar2/labelmap_s.prototxt')
file = open(labelmap_file, 'r')
labelmap = caffe_pb2.LabelMap()
text_format.Merge(str(file.read()), labelmap)

def get_labelname(labelmap, labels):
    num_labels = len(labelmap.item)
    labelnames = []
    if type(labels) is not list:
        labels = [labels]
    for label in labels:
        found = False
        for i in xrange(0, num_labels):
            if label == labelmap.item[i].label:
                found = True
                labelnames.append(labelmap.item[i].display_name)
                break
        assert found == True
    return labelnames

* Load the net in the test phase for inference, and configure input preprocessing.

In [ ]:
model_def = 'caffe/models/VGGNet/nexar2/SSD_600x600/deploy.prototxt'
model_weights = 'caffe/models/VGGNet/nexar2/SSD_600x600/VGG_nexar2_SSD_600x600_iter_10000.caffemodel'
#model_def = '/home/user/public/nexar/SSD/VGG16/13/deploy.prototxt'
#model_weights = '/home/user/public/nexar/SSD/VGG16/13/VGG_nexar2_SSD_600x600_iter_10000.caffemodel'

net = None
net = caffe.Net(model_def,      # defines the structure of the model
                model_weights,  # contains the trained weights
                caffe.TEST)     # use test mode (e.g., don't perform dropout)

input_shape = None

if input_shape is not None:
    net.blobs['data'].reshape(1, 3, input_shape[0], input_shape[1])

# input preprocessing: 'data' is the name of the input blob == net.inputs[0]
transformer = caffe.io.Transformer({'data': net.blobs['data'].data.shape})
transformer.set_transpose('data', (2, 0, 1))
transformer.set_mean('data', np.array([104,117,123])) # mean pixel
transformer.set_raw_scale('data', 255)  # the reference model operates on images in [0,255] range instead of [0,1]
transformer.set_channel_swap('data', (2,1,0))  # the reference model has channels in BGR order instead of RGB

### 2. SSD detection

* Load an image.

In [ ]:
image = caffe.io.load_image('examples/8.jpg')
plt.imshow(image)
plt.axis('off');

* Run the net and examine the top_k results

In [ ]:
transformed_image = transformer.preprocess('data', image)
net.blobs['data'].data[...] = transformed_image

# Forward pass.
detections = net.forward()['detection_out']

# Parse the outputs.
det_conf = detections[0,0,:,2]
det_xmin = detections[0,0,:,3]
det_ymin = detections[0,0,:,4]
det_xmax = detections[0,0,:,5]
det_ymax = detections[0,0,:,6]

conf_thresh = 0.3

# Get detections with confidence higher than some threshold.
top_indices = []
for i in range(det_conf.shape[0]):
    if det_conf[i] >= conf_thresh:
        top_indices.append(i)

top_conf = det_conf[top_indices]
top_xmin = det_xmin[top_indices]
top_ymin = det_ymin[top_indices]
top_xmax = det_xmax[top_indices]
top_ymax = det_ymax[top_indices]

# Plot the boxes

plt.figure(figsize=(15,10))
plt.imshow(image)
plt.axis('off')
currentAxis = plt.gca()

for i in xrange(top_conf.shape[0]):
    xmin = int(round(top_xmin[i] * image.shape[1]))
    ymin = int(round(top_ymin[i] * image.shape[0]))
    xmax = int(round(top_xmax[i] * image.shape[1]))
    ymax = int(round(top_ymax[i] * image.shape[0]))
    score = top_conf[i]
    display_txt = '%s: %.2f'%('car', score)
    coords = (xmin, ymin), xmax-xmin+1, ymax-ymin+1
    currentAxis.add_patch(plt.Rectangle(*coords, fill=False, edgecolor=[1,0,0], linewidth=2))
    currentAxis.text(xmin, ymin, display_txt, bbox={'facecolor':[1,0,0], 'alpha':0.5})
    
    #print '%s: %.2f'%(label_name, score)

## Evaluate on Nexar validation set

In [ ]:
import pandas as pd
from collections import defaultdict

# path to the validation images folder
val_image_root = os.path.abspath('./data/train')

val_anno_df = pd.read_csv('val.csv')

In [ ]:
max_val_num = 0

val_names = list(set(val_anno_df['image_filename']))

det_dict = defaultdict(list)

conf_thresh = 0.05

coord_func = lambda x: round(x)

max_val_num = max_val_num if max_val_num > 0 else len(val_names)
for iname in range(max_val_num):
    # load & transform image
    name = val_names[iname]
    image_path = os.path.join(val_image_root, name)
    image = caffe.io.load_image(image_path)
    transformed_image = transformer.preprocess('data', image)
    
    # detect
    net.blobs['data'].data[...] = transformed_image
    detections = net.forward()['detection_out']
    
    det_conf = detections[0,0,:,2]
    det_xmin = detections[0,0,:,3]
    det_ymin = detections[0,0,:,4]
    det_xmax = detections[0,0,:,5]
    det_ymax = detections[0,0,:,6]

    conf_mask = det_conf >= conf_thresh

    h, w = image.shape[:2]
    for ind in np.nonzero(conf_mask)[0]:
        det_dict['image_filename'].append(name)
        x0 = coord_func(w * det_xmin[ind])
        y0 = coord_func(h * det_ymin[ind])
        x1 = coord_func(w * det_xmax[ind])
        y1 = coord_func(h * det_ymax[ind])
        det_dict['x0'].append(x0)
        det_dict['y0'].append(y0)
        det_dict['x1'].append(x1)
        det_dict['y1'].append(y1)        
        det_dict['label'].append('car')
        det_dict['confidence'].append(det_conf[ind])
        
    if iname > 0 and iname % 100 == 0: 
        print('%s/%s'%(iname,max_val_num))
        
    if iname == max_val_num - 1:
        break
        
val_det_df = pd.DataFrame(det_dict, columns=['image_filename','x0','y0','x1','y1','label','confidence'])
val_det_df.to_csv('val_det.csv', index=False)

print('Done')

In [ ]:
!python challenge2-evaluation/evaluate/eval_challenge.py -g ./val.csv -d ./val_det.csv